In [7]:
import numpy as np
import scipy.sparse as sps
import scipy as sp
from matplotlib import pyplot as plt



In [8]:
# create numpy array
R = np.array([[1,-1,1,-1,1,-1],[1,1,0,-1,-1,-1],[0,1,1,-1,-1,0],[-1,-1,-1,1,1,1],[-1,0,-1,1,1,1]])
R

array([[ 1, -1,  1, -1,  1, -1],
       [ 1,  1,  0, -1, -1, -1],
       [ 0,  1,  1, -1, -1,  0],
       [-1, -1, -1,  1,  1,  1],
       [-1,  0, -1,  1,  1,  1]])

In [9]:
Rs = sps.coo_array(R)
Rsc = Rs.tocsc()
Rsr = Rs.tocsr()

In [10]:
#
num_users = 5
num_items = 6

#
nnz_entries_per_col = np.diff(Rsc.indptr)
nnz_entries_per_row = np.diff(Rsr.indptr)

nonzero_rows = Rsc.nonzero()[0]
nonzero_cols = Rsc.nonzero()[1]



In [16]:
# implement Naive Bayes model with Laplace smoothing

alpha = 1
scoring_options = [-1,1]
num_scoring_options = len(scoring_options)
new_movie_id = np.array(range(num_items))

rfin =[]
# loop over all users 
for juser in range(num_users): 
    juser_rated = Rsr.indices[Rsr.indptr[juser]:Rsr.indptr[juser+1]] 
    to_rate = np.delete(new_movie_id,juser_rated)

    # loop over all items that user did not rate
    for jitem in to_rate: 
        rpred = []

        # compute probability of rating for all possible ratings  
        for jprob in scoring_options:
            jaa = Rsc.data[Rsc.indptr[jitem]:Rsc.indptr[jitem+1]] == jprob           
            aa = Rsc.indices[Rsc.indptr[jitem]:Rsc.indptr[jitem+1]][jaa]  # all users that rated item jitem as jprob
            prior = (np.count_nonzero(jaa) + alpha)/(nnz_entries_per_col[jitem] + num_scoring_options*alpha)       
            Pr = 1

            # loop over items that juser rated   
            for j in juser_rated:                          
                ind_j0 = Rsc.indptr[j]
                ind_j1 = Rsc.indptr[j+1]
                bb = Rsc.indices[ind_j0:ind_j1] # all users that rated item j
                cc = Rsc.data[ind_j0:ind_j1]          
                bp = np.isin(bb,aa,assume_unique=True)         
                Pr = Pr * (np.count_nonzero(cc[bp] == cc[bb == juser]) + alpha)/ (np.count_nonzero(bp) + num_scoring_options*alpha)

            rpred.append(prior*Pr)
        
        rfin.append(rpred)

np.round(rfin,5)


array([[0.00065, 0.04167],
       [0.0026 , 0.0625 ],
       [0.0625 , 0.0026 ],
       [0.02344, 0.00116]])